In [1]:
import requests
import pandas as pd

# Indicadores a serem coletados
indicators = {
    "Population total": "SP.POP.TOTL",
    "Crescimento Populacional %":"SP.POP.GROW",
    "Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population)": "SI.POV.DDAY",
    "Gini index": "SI.POV.GINI",
    "GDP (current US$)": "NY.GDP.MKTP.CD",
    "Foreign direct investment, net inflows (BoP, current US$)": "BX.KLT.DINV.CD.WD",
    "Unemployment, total (% of total labor force) (modeled ILO estimate)": "SL.UEM.TOTL.ZS",
    "Internet users (% of population)": "IT.NET.USER.ZS",
    "Mobile cellular subscriptions (per 100 people)": "IT.CEL.SETS.P2",
    "Fixed broadband subscriptions (per 100 people)": "IT.NET.SECR.P6",
    "GNI (current US$)": "NY.GNP.MKTP.CD",
    "Urban population (% of total population)": "SP.URB.TOTL.IN.ZS",
    "Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)": "SL.TLF.CACT.ZS",
    "Employment to population ratio, 15+, total (%) (modeled ILO estimate)": "SL.EMP.TOTL.SP.ZS",
    "Stocks traded, total value (% of GDP)": "CM.MKT.TRAD.GD.ZS",
    "Domestic credit to private sector (% of GDP)": "FS.AST.PRVT.GD.ZS",
    "GDP growth (annual %)": "NY.GDP.MKTP.KD.ZG",
    "Gross fixed capital formation (annual % growth)": "NE.GDI.FTOT.KD.ZG",
    "Employment growth (annual %)": "SL.EMP.TOTL.SP.ZG",
    "Youth unemployment (% of total labor force ages 15-24) (modeled ILO estimate)": "SL.UEM.1524.ZS",
    "Labor force with tertiary education (% of total)": "SL.TLF.TERT.ZS"
}

# Função para obter dados da API do Banco Mundial
def get_indicator_data(country_code, indicator_code):
    url = f"http://api.worldbank.org/v2/country/{country_code}/indicator/{indicator_code}?format=json&per_page=10000"
    response = requests.get(url)
    data = response.json()
    if isinstance(data, list) and len(data) > 1:
        return data[1]
    return []

# Coletar dados para Argentina
country_code = 'AR'
all_data = {}
for description, code in indicators.items():
    all_data[description] = get_indicator_data(country_code, code)

# Criar DataFrame
df = pd.DataFrame()
for description in all_data:
    if all_data[description]:
        temp_df = pd.DataFrame(all_data[description])[['date', 'value']]
        temp_df = temp_df.rename(columns={'value': description})
        if df.empty:
            df = temp_df
        else:
            df = pd.merge(df, temp_df, on='date', how='outer')

# Converter a coluna date para datetime
df['date'] = pd.to_datetime(df['date'], format='%Y')

# Ordenar DataFrame por data
df = df.sort_values(by='date')

# Função para formatar os números
def format_number(x):
    if pd.isna(x):
        return x
    if isinstance(x, (int, float)):
        if abs(x) >= 1e6:
            return f"{x:,.2e}"  # Notação científica para valores muito grandes
        else:
            return f"{x:,.2f}"  # Arredondar para duas casas decimais
    return x

# Aplicar a formatação
formatted_df = df.applymap(format_number)
formatted_df = formatted_df.fillna(0.00)
formatted_df.to_csv('argentina_2.csv')
# Exibir os primeiros registros do DataFrame formatado
formatted_df


,date,Population total,Crescimento Populacional %,Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population),Gini index,GDP (current US$),"Foreign direct investment, net inflows (BoP, current US$)","Unemployment, total (% of total labor force) (modeled ILO estimate)",Internet users (% of population),Mobile cellular subscriptions (per 100 people),Fixed broadband subscriptions (per 100 people),GNI (current US$),Urban population (% of total population),"Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)","Employment to population ratio, 15+, total (%) (modeled ILO estimate)","Stocks traded, total value (% of GDP)",Domestic credit to private sector (% of GDP),GDP growth (annual %),Gross fixed capital formation (annual % growth),Youth unemployment (% of total labor force ages 15-24) (modeled ILO estimate)
63,1960-01-01,2.03e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,73.61,0.0,0.0,0.0,13.61,0.0,0.0,0.0
62,1961-01-01,2.07e+07,1.61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.22,0.0,0.0,0.0,14.09,5.43,14.17,0.0
61,1962-01-01,2.10e+07,1.63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.77,0.0,0.0,0.0,12.01,-0.85,-8.03,0.0
60,1963-01-01,2.14e+07,1.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.31,0.0,0.0,0.0,11.52,-5.31,-13.49,0.0
59,1964-01-01,2.17e+07,1.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.84,0.0,0.0,0.0,10.63,10.13,10.09,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2019-01-01,4.49e+07,0.99,1.10,43.30,4.48e+11,6.65e+09,9.84,79.95,125.94,"3,018.49",4.30e+11,91.99,60.29,54.36,0.67,0.0,-2.00,-15.95,25.81
3,2020-01-01,4.54e+07,0.97,1.20,42.70,3.86e+11,4.88e+09,11.46,85.51,121.60,"3,685.76",3.75e+11,92.11,56.02,49.60,0.74,0.0,-9.90,-13.07,30.41
2,2021-01-01,4.58e+07,0.95,0.90,42.40,4.88e+11,6.90e+09,8.74,87.15,130.46,0.0,4.78e+11,92.23,60.36,55.09,0.54,0.0,10.72,33.80,23.37
1,2022-01-01,4.62e+07,0.93,0.60,40.70,6.31e+11,1.54e+10,6.80,88.38,132.36,0.0,6.19e+11,92.35,60.47,56.35,1.26,0.0,4.96,11.06,18.98


1. **Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population)** (`SI.POV.DDAY`)
   - **Descrição**: Percentual da população vivendo com menos de $1.90 por dia, ajustado pela Paridade do Poder de Compra (PPP) de 2011. Este indicador mede a extrema pobreza.
   
2. **Gini index** (`SI.POV.GINI`)
   - **Descrição**: Medida da desigualdade de renda dentro de um país. O índice varia de 0 a 100, onde 0 indica perfeita igualdade (todos têm a mesma renda) e 100 indica perfeita desigualdade (uma pessoa tem toda a renda, e todos os outros não têm nenhuma).

3. **GDP (current US$)** (`NY.GDP.MKTP.CD`)
   - **Descrição**: Produto Interno Bruto em valores correntes, medido em dólares americanos. Representa o valor total de todos os bens e serviços produzidos dentro de um país em um determinado ano.

4. **Foreign direct investment, net inflows (BoP, current US$)** (`BX.KLT.DINV.CD.WD`)
   - **Descrição**: Investimento direto estrangeiro líquido, que inclui o valor dos investimentos feitos por entidades estrangeiras no país, menos o valor dos investimentos feitos por entidades do país no exterior, medido em dólares americanos correntes.

5. **Unemployment, total (% of total labor force) (modeled ILO estimate)** (`SL.UEM.TOTL.ZS`)
   - **Descrição**: Taxa de desemprego total como percentual da força de trabalho total. Este é um valor estimado pelo modelo da Organização Internacional do Trabalho (OIT).

6. **Internet users (% of population)** (`IT.NET.USER.ZS`)
   - **Descrição**: Percentual da população que usa a internet. Este indicador mede a penetração da internet entre a população.

7. **Mobile cellular subscriptions (per 100 people)** (`IT.CEL.SETS.P2`)
   - **Descrição**: Número de assinaturas de telefonia móvel celular por 100 pessoas. Inclui assinaturas pós-pagas e pré-pagas.

8. **Fixed broadband subscriptions (per 100 people)** (`IT.NET.SECR.P6`)
   - **Descrição**: Número de assinaturas de banda larga fixa por 100 pessoas. Inclui conexões de alta velocidade como DSL, cabo, fibra óptica, entre outras.

9. **GNI (current US$)** (`NY.GNP.MKTP.CD`)
   - **Descrição**: Renda Nacional Bruta em valores correntes, medida em dólares americanos. Representa o valor total dos bens e serviços produzidos por residentes de um país, incluindo receitas recebidas do exterior.

10. **Urban population (% of total population)** (`SP.URB.TOTL.IN.ZS`)
   - **Descrição**: Percentual da população total que vive em áreas urbanas. Este indicador mede o grau de urbanização de um país.

11. **Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)** (`SL.TLF.CACT.ZS`)
   - **Descrição**: Taxa de participação na força de trabalho total como percentual da população total com 15 anos ou mais. Este é um valor estimado pelo modelo da OIT.

12. **Employment to population ratio, 15+, total (%) (modeled ILO estimate)** (`SL.EMP.TOTL.SP.ZS`)
   - **Descrição**: Razão de emprego para a população total com 15 anos ou mais, como percentual. Este indicador mede a proporção de pessoas empregadas na população adulta, estimado pelo modelo da OIT.

13. **Stocks traded, total value (% of GDP)** (`CM.MKT.TRAD.GD.ZS`)
   - **Descrição**: Valor total das ações negociadas como percentual do PIB. Mede a liquidez do mercado de ações, representando o valor total das ações negociadas em um ano em relação ao PIB.

14. **Domestic credit to private sector (% of GDP)** (`FS.AST.PRVT.GD.ZS`)
   - **Descrição**: Crédito doméstico ao setor privado como percentual do PIB. Inclui os recursos financeiros fornecidos ao setor privado, como empréstimos e créditos.
